
# API TWITTER 
Objective:
This api has the objective to collect data from stock market inside a tweet api.
The focus of the API is to process info for IBOVESPA market.

<h1> 1. Collect and process data inside a mongoDB database </h1>

# 1.1 Import packages

In [10]:
#Api packages
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
#packages for data manipulation
from datetime import datetime
import json
import pickle
import pandas as pd
#packages for storage
from pymongo import MongoClient

In [37]:
#extract the api credentials for pickles files
api_key_df = pd.read_pickle(r'crendentials_key.pkl')
api_token_df = pd.read_pickle(r'crendentials_token.pkl')

In [54]:
access_token = '725464864861331456-Dj9KxUbWuTjjES6fU11YJoJhqfWSoHb'
api_key = 'V8bym5SOBxQziEJUiyv63725L'
api_key_secret = 'CatdyRop1VLZQ4uCHKwZh8bMI6B3w46qtsOlREGrbpmtATOuP7'
access_token='725464864861331456-Dj9KxUbWuTjjES6fU11YJoJhqfWSoHb'
access_token_secret= 'ElRX10ABnyU8jabnMfZK5utMkTOIaom9oV2KRktDNSkQM'

In [56]:
#Connect with the Twitter API
auth = OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [57]:
auth

In [58]:
#Connect with the mongodb
conn = MongoClient('localhost',27017)
print(conn)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [59]:
api_token_df

['access_token=725464864861331456-Dj9KxUbWuTjjES6fU11YJoJhqfWSoHb',
 'access_token_secret=ElRX10ABnyU8jabnMfZK5utMkTOIaom9oV2KRktDNSkQM']

In [60]:
#create a database
db = conn.twitterdb

In [61]:
#input colls inside the database
col = db.tweets

In [62]:
print(api_key_df)

['api_key=V8bym5SOBxQziEJUiyv63725L', 'api_key_secret=CatdyRop1VLZQ4uCHKwZh8bMI6B3w46qtsOlREGrbpmtATOuP7']


# 2. Collect Data from Tweet API

In [63]:
#Create a Class for get the tweets data
class Twitter_Collector(StreamListener):
    def on_data(self, data):
        tweet = json.loads(data)
        created_at = tweet['created_at']
        id_str = tweet['id']
        tweet_text = tweet['text']
        obj =  {"created_at":created_at,"id_str":id_str,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [64]:
ttcc = Twitter_Collector() 

In [65]:
stream = Stream(auth, listener= ttcc)

In [66]:
keywords = ['Ibovespa','bovespa','MXRF11','fundos_imobiliários']

In [69]:
stream.filter(track=keywords)

TweepError: Stream object already connected!

In [ ]:
dataset = [[]]